## Work

Idea:
- Represent folders as objects
- After basic input cleaning, parse the inputs one command at a time into a tree of Folder objects
- Iterate through the folders in the tree to figure out their sizes
- Do the final math on the folder file sizes

### 1

In [57]:
# Folder objects to keep track of things
class Folder():
    def __init__(self, name, parent):
        # Add random numbers on the name to ensure names are unique, lmao
        self.name = name + '_' + str(np.random.random())
        self.parent = parent
        self.files = {}
        self.size = None
    
    def set_files(self, files):
        for item in files:
            parts = item.split()
            # Only record if it's not already there...to avoid issues with visiting a folder twice
            # Don't think this matters, but it makes me feel good
            if not parts[1] in self.files:
                # Record a folder
                if parts[0] == 'dir':
                    self.files[parts[1]] = Folder(parts[1], self)
                # Record a file
                else:
                    self.files[parts[1]] = int(parts[0])
    
    def navigate(self, folder):
        if folder == '..':
            return self.parent
        else:
            assert folder in self.files
            return self.files[folder]
    
    def get_size(self):
        if self.size is None:
            self.size = 0
            for obj in self.files.values():
                # Folder
                if isinstance(obj, Folder):
                    self.size += obj.get_size()
                # File
                else:
                    self.size += obj
        return self.size
    
    def __str__(self):
        return str({key : str(value) for key, value in self.files.items()})

In [58]:
# Parse terminal output into a bunch of nested dictionaries
# Dict = folder
# Key = file (so value is size) or folder (so value is dict)
# I checked and there's just one $cd / at the start, so no need to worry about that
# And I'll have a dictionary item '..' which gives the dictionary a level up, for moving up through folders
def parse_terminal(terminal):
    # Set up the tree and our current view of it
    tree = Folder('/', None)
    node = tree
    
    # Iterate over commands
    for command in terminal:
        
        # ls
        if command[0] == 'ls':
            node.set_files(command[1:])
                    
        # cd
        else:
            folder = command[0].split()[1]
            node = node.navigate(folder)

    return tree

# Iterate through the tree and get a dictionary of each folder with its size
def parse_tree(tree):
    # Calculate main folder size
    sizes = {'/' : tree.get_size()}
    
    # Iterate through subfolders
    queue = list(tree.files.values())
    while queue:
        obj = queue.pop()
        if isinstance(obj, Folder):
            for item in obj.files.values():
                queue.append(item)
            sizes[obj.name] = obj.get_size()
    
    return sizes

In [82]:
# Format the input nicely
terminal = [[string for string in block if string != ''] for block in split(t, block_char = '$ ')[2:]]
terminal

[['ls', 'dir a', '14848514 b.txt', '8504156 c.dat', 'dir d'],
 ['cd a'],
 ['ls', 'dir e', '29116 f', '2557 g', '62596 h.lst'],
 ['cd e'],
 ['ls', '584 i'],
 ['cd ..'],
 ['cd ..'],
 ['cd d'],
 ['ls', '4060174 j', '8033020 d.log', '5626152 d.ext', '7214296 k']]

In [83]:
# Parse into objects
tree = parse_terminal(terminal)
print(tree)

{'a': '{\'e\': "{\'i\': \'584\'}", \'f\': \'29116\', \'g\': \'2557\', \'h.lst\': \'62596\'}', 'b.txt': '14848514', 'c.dat': '8504156', 'd': "{'j': '4060174', 'd.log': '8033020', 'd.ext': '5626152', 'k': '7214296'}"}


In [84]:
# Parse into sizes
sizes_t = parse_tree(tree)
sizes_t

{'/': 48381165,
 'd_0.9702901950017301': 24933642,
 'a_0.34593574882718403': 94853,
 'e_0.3498957569662525': 584}

In [85]:
# The answer
sum([val for val in sizes_t.values() if val <= 100000])

95437

In [97]:
# Real run
terminal = [[string for string in block if string != ''] for block in split(s, block_char = '$ ')[2:]]
tree = parse_terminal(terminal)
sizes_s = parse_tree(tree)
sum([val for val in sizes_s.values() if val <= 100000])

1077191

Bugs I had to fix:
- Was dropping the last file name in the input, so my answer was too small
- Folder names overlapped so some would get overwritten in the folder sizes dictionary (sizes_s), so my answer was too small

### 2

In [88]:
# Need to free up this much space
drop_t = sizes_t['/'] - 40000000
drop_t

8381165

In [89]:
# Sizes of valid directories to delete that would get the job done
[size for size in sizes_t.values() if size >= drop_t]

[48381165, 24933642]

In [92]:
# Actual run
drop_s = sizes_s['/'] - 40000000
min([size for size in sizes_s.values() if size >= drop_s])

5649896

In [95]:
sizes_s['/']

45174025

In [93]:
drop_s

5174025

## Utilities

In [5]:
# Remove initial/final \n characters
def clean(s):
    return s[1:-1]

# Split at \n characters
# If there are \n\n characters, split into blocks too
def split(s, block_char = '\n\n', line_char = '\n'):
    out = [block.split(line_char) for block in clean(s).split(block_char)]
    if len(out) == 1:
        return out[0]
    else:
        return out

# Apply a function(s) to a list or "block" data (2-level list)
def apply_func(data, func, nested=False):
    if not isinstance(func, list):
        func = [func]
        
    def _func(x):
        for f in func:
            x = f(x)
        return x
        
    if nested:
        return [[_func(x) for x in block] for block in data]
    else:
        return [_func(x) for x in data]

# Split, parsing everything as ints
def split_int(s):
    return apply_func(split(s), int)

# Split, parsing everything as float
def split_float(s):
    return apply_func(split(s), float)

## Input

In [67]:
t = """
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
"""

In [7]:
s = """
$ cd /
$ ls
dir cmjgvh
dir czrzl
dir fcbt
dir hdh
259661 hjsbd.mzp
dir jgrdd
dir lqblqtng
dir pgvmpmn
dir pqqcvcm
dir zglbptq
$ cd cmjgvh
$ ls
dir hdh
134565 hdh.sjv
dir hgrpfmt
282147 mjtq.ffd
42343 rvmzv.rtb
dir sjgvbd
31468 wgtjmb.thf
$ cd hdh
$ ls
267125 htplc.gdw
$ cd ..
$ cd hgrpfmt
$ ls
39132 lndwz
280595 rffmsvdw
$ cd ..
$ cd sjgvbd
$ ls
26464 ghg.zmq
1533 zsgdbd.dmm
$ cd ..
$ cd ..
$ cd czrzl
$ ls
dir cmh
242795 hpgnd.mmt
157748 hps.ptg
129797 qjrhbjql.zdc
18290 sfhrzzcd.hwm
63141 sngg.vdw
$ cd cmh
$ ls
dir nglsj
dir szs
76775 vdpqhvrm.mcz
$ cd nglsj
$ ls
307082 cjvph.fvc
286825 szs.vpj
$ cd ..
$ cd szs
$ ls
6233 fgpnnvm
94070 jjc.szq
$ cd ..
$ cd ..
$ cd ..
$ cd fcbt
$ ls
150417 grncq.brq
$ cd ..
$ cd hdh
$ ls
dir cljdmh
dir fptsr
dir schjz
dir vrrcrhzs
$ cd cljdmh
$ ls
dir blsrsqz
21876 blthtcl
287047 cjvph.fvc
dir grncq
dir hdh
dir jdjps
dir mgzlprt
dir mrwc
33008 pcwnlp.czm
dir rdtdjb
135650 smf
173978 zsgdbd.dmm
$ cd blsrsqz
$ ls
275130 dvqsffcn
$ cd ..
$ cd grncq
$ ls
213418 bhdpmv.zzt
153483 grncq.jsf
$ cd ..
$ cd hdh
$ ls
dir dzrrh
dir gjjqhfq
231302 grncq
dir hdh
dir jnfczjjt
dir ljl
dir pwv
154659 rqvgf
dir vdvg
dir vpdqnv
$ cd dzrrh
$ ls
dir grncq
$ cd grncq
$ ls
288753 grd
$ cd ..
$ cd ..
$ cd gjjqhfq
$ ls
dir hdh
dir mnj
$ cd hdh
$ ls
241795 szs.mhq
$ cd ..
$ cd mnj
$ ls
200772 sfhrzzcd.hwm
237342 zsgdbd.dmm
$ cd ..
$ cd ..
$ cd hdh
$ ls
30588 qptfqt.ggc
$ cd ..
$ cd jnfczjjt
$ ls
93796 rqbsqhp.grv
30301 sfhrzzcd.hwm
$ cd ..
$ cd ljl
$ ls
dir lvpvmlnb
dir tsb
227393 twnj
$ cd lvpvmlnb
$ ls
16586 gnlmdb
340921 grncq
dir lqblqtng
dir nvp
$ cd lqblqtng
$ ls
190888 dfjrnbwq
79149 dvqsffcn
dir grncq
267567 lqblqtng
169475 vzwm.pnd
305249 zsgdbd.dmm
$ cd grncq
$ ls
323438 zjpg
$ cd ..
$ cd ..
$ cd nvp
$ ls
239588 mffsww.qzc
dir qqhwjn
208451 qrjv.lns
34563 sfhrzzcd.hwm
$ cd qqhwjn
$ ls
47343 wcpqgvh.cfl
$ cd ..
$ cd ..
$ cd ..
$ cd tsb
$ ls
dir lqblqtng
40229 szs
214138 whfw
$ cd lqblqtng
$ ls
16005 fgpnnvm
$ cd ..
$ cd ..
$ cd ..
$ cd pwv
$ ls
dir hzwljqm
51565 pjvpm
$ cd hzwljqm
$ ls
106132 fgpnnvm
$ cd ..
$ cd ..
$ cd vdvg
$ ls
267870 zgm.wbw
$ cd ..
$ cd vpdqnv
$ ls
dir cmmc
239093 dvqsffcn
dir rhgb
142265 sfhrzzcd.hwm
dir szs
$ cd cmmc
$ ls
dir pjhhbggb
$ cd pjhhbggb
$ ls
195797 psqml.cjl
$ cd ..
$ cd ..
$ cd rhgb
$ ls
102805 qrnntf
$ cd ..
$ cd szs
$ ls
314078 tcslpbc
$ cd ..
$ cd ..
$ cd ..
$ cd jdjps
$ ls
12330 twnj
$ cd ..
$ cd mgzlprt
$ ls
15552 gzthzjsr
dir hdh
dir qrfvn
dir twnj
331113 zsgdbd.dmm
$ cd hdh
$ ls
dir cfdrnjsg
343470 zsgdbd.dmm
$ cd cfdrnjsg
$ ls
dir gddbd
309726 grncq.gmr
dir vzdfj
$ cd gddbd
$ ls
226040 cjvph.fvc
47672 twnj
67109 zsgdbd.dmm
$ cd ..
$ cd vzdfj
$ ls
29653 grncq
$ cd ..
$ cd ..
$ cd ..
$ cd qrfvn
$ ls
74136 lqblqtng
dir mjjrz
dir zrsz
$ cd mjjrz
$ ls
27688 fgpnnvm
$ cd ..
$ cd zrsz
$ ls
9100 wbrgdtv
$ cd ..
$ cd ..
$ cd twnj
$ ls
41216 cjvph.fvc
250320 wnhjfm
$ cd ..
$ cd ..
$ cd mrwc
$ ls
dir bjwnmw
116091 grncq.phw
24868 hdh.lwn
29567 qsrtrvr.jbw
100251 szs.dvg
$ cd bjwnmw
$ ls
128769 rdpsnm
$ cd ..
$ cd ..
$ cd rdtdjb
$ ls
132588 gsgjr
149600 lqblqtng.nnr
179302 wfbqblml.tgc
76170 wfrcm.fvp
$ cd ..
$ cd ..
$ cd fptsr
$ ls
169594 lzlcml.mgm
dir zrc
dir ztd
$ cd zrc
$ ls
110339 gjpgwrcm.lhg
$ cd ..
$ cd ztd
$ ls
203770 zsgdbd.dmm
$ cd ..
$ cd ..
$ cd schjz
$ ls
3212 cjvph.fvc
288619 jcltshwj
295116 qmbp.mpd
$ cd ..
$ cd vrrcrhzs
$ ls
dir twnj
$ cd twnj
$ ls
70492 dvqsffcn
44411 mhgspcgz
$ cd ..
$ cd ..
$ cd ..
$ cd jgrdd
$ ls
325202 dvqsffcn
$ cd ..
$ cd lqblqtng
$ ls
dir bbzv
dir gzqqp
dir vgt
$ cd bbzv
$ ls
326190 fgpnnvm
$ cd ..
$ cd gzqqp
$ ls
179929 crn.vpf
263365 dvqsffcn
251298 hdh.rsh
dir jgrtqpv
62514 lqblqtng.spm
dir njdhqsvj
32894 sfhrzzcd.hwm
$ cd jgrtqpv
$ ls
dir bmvfszlz
286549 zsgdbd.dmm
$ cd bmvfszlz
$ ls
169838 fgpnnvm
$ cd ..
$ cd ..
$ cd njdhqsvj
$ ls
2729 bgd
294119 fgcfbrdz.bll
$ cd ..
$ cd ..
$ cd vgt
$ ls
dir btf
dir csgvtvsq
dir czbhw
dir jqnvscr
dir nqp
dir rlvnnmh
dir tmf
dir twjtnhll
$ cd btf
$ ls
333891 cjvph.fvc
$ cd ..
$ cd csgvtvsq
$ ls
dir ccrvn
dir szs
dir tsmqm
dir tssfbq
$ cd ccrvn
$ ls
14614 qbl.rjg
$ cd ..
$ cd szs
$ ls
92863 vswznzs
$ cd ..
$ cd tsmqm
$ ls
206954 dncrjt.sch
252714 zsgdbd.dmm
64306 zzr.snv
$ cd ..
$ cd tssfbq
$ ls
326310 chffstg.qcr
51533 cjvph.fvc
$ cd ..
$ cd ..
$ cd czbhw
$ ls
290187 bdn.vjp
185832 cjvph.fvc
dir lqblqtng
198843 twnj.flf
64179 zjwg.jwc
256134 zrtmfn
$ cd lqblqtng
$ ls
82035 cjvph.fvc
$ cd ..
$ cd ..
$ cd jqnvscr
$ ls
dir bgqf
255437 dvqsffcn
147384 fzwzqb
dir gbf
dir gqzf
dir grncq
287135 grncq.pcg
333854 hqw.hgc
dir nzrsswd
dir svmqtq
$ cd bgqf
$ ls
325037 jsv
$ cd ..
$ cd gbf
$ ls
274259 fgpnnvm
$ cd ..
$ cd gqzf
$ ls
224386 hwwqrq.hcz
$ cd ..
$ cd grncq
$ ls
dir bvsfn
dir gcbzghtz
100460 njbgfrg
210677 phnslpqc.rbf
dir tgz
95034 vdpbwsn
dir vsrnbb
$ cd bvsfn
$ ls
184581 lwj.gvm
dir pszwl
129220 sfhrzzcd.hwm
162987 smdvbnrq.zjd
71207 tspgccr.gdf
323030 zsgdbd.dmm
$ cd pszwl
$ ls
dir szs
dir twnj
$ cd szs
$ ls
341367 cbcvvtg.hcg
112908 rggrhm
$ cd ..
$ cd twnj
$ ls
29609 sfhrzzcd.hwm
$ cd ..
$ cd ..
$ cd ..
$ cd gcbzghtz
$ ls
139868 scggzqr.bzw
268881 smlhjltf.rwr
247122 wbwzdpg.djs
$ cd ..
$ cd tgz
$ ls
214842 grncq
dir njffnbp
dir sfvsdzs
dir sslvmwt
212789 szqph.gmw
$ cd njffnbp
$ ls
243579 lmwd.mgz
$ cd ..
$ cd sfvsdzs
$ ls
7846 nthbtv.zdp
$ cd ..
$ cd sslvmwt
$ ls
241215 szs.rlb
$ cd ..
$ cd ..
$ cd vsrnbb
$ ls
69152 cjvph.fvc
dir dvjmd
142844 dvqsffcn
242145 msm.nlf
dir nttvm
dir rjfjwbsm
47597 twnj.lft
123883 zsgdbd.dmm
$ cd dvjmd
$ ls
dir lnhr
70772 qvs.gmv
80548 twnj.jdf
$ cd lnhr
$ ls
202959 hdh
$ cd ..
$ cd ..
$ cd nttvm
$ ls
dir hdh
$ cd hdh
$ ls
178915 fzgzdpbv.jdj
$ cd ..
$ cd ..
$ cd rjfjwbsm
$ ls
114145 grncq
$ cd ..
$ cd ..
$ cd ..
$ cd nzrsswd
$ ls
dir bnrj
17967 cjvph.fvc
237712 fgpnnvm
dir spqqqd
dir szs
96651 twnj
6447 twnj.dft
$ cd bnrj
$ ls
40924 cbhq.wpc
175930 cjvph.fvc
dir grncq
dir lfrtwv
267323 mmq.zwz
dir qsbmrs
dir wpsj
$ cd grncq
$ ls
151440 ctmwl
83350 lqblqtng
145599 tpqfd.zfv
$ cd ..
$ cd lfrtwv
$ ls
dir cdzvst
144562 fgpnnvm
dir pllhlhr
dir qvdlrsw
$ cd cdzvst
$ ls
211524 dvsnph.hrf
dir wchrhl
$ cd wchrhl
$ ls
73032 lntqbfl.nbz
$ cd ..
$ cd ..
$ cd pllhlhr
$ ls
70197 dtccz
$ cd ..
$ cd qvdlrsw
$ ls
dir gdmppfzv
$ cd gdmppfzv
$ ls
244356 nhnsd
$ cd ..
$ cd ..
$ cd ..
$ cd qsbmrs
$ ls
155216 cvt.bdn
dir gcvlpg
dir hdh
$ cd gcvlpg
$ ls
306542 bwnfl.rsl
67067 cjvph.fvc
341913 zsgdbd.dmm
$ cd ..
$ cd hdh
$ ls
138120 cjvph.fvc
$ cd ..
$ cd ..
$ cd wpsj
$ ls
301391 sfhrzzcd.hwm
$ cd ..
$ cd ..
$ cd spqqqd
$ ls
40806 dvqsffcn
dir gqsbsmfm
41886 grncq.jqr
dir pdfsb
44167 pnq.nfr
127242 twnj.chn
125868 vvgzv.rmv
$ cd gqsbsmfm
$ ls
99936 dvqsffcn
$ cd ..
$ cd pdfsb
$ ls
253190 fgpnnvm
$ cd ..
$ cd ..
$ cd szs
$ ls
164955 vswnnw
$ cd ..
$ cd ..
$ cd svmqtq
$ ls
dir grncq
$ cd grncq
$ ls
274192 hjdfj.qnw
dir tcrhb
dir twnj
$ cd tcrhb
$ ls
64293 grncq.jvh
$ cd ..
$ cd twnj
$ ls
149048 cjvph.fvc
192053 rdgv
101473 zct.rmf
280169 zsgdbd.dmm
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd nqp
$ ls
107620 ndgz.gwb
$ cd ..
$ cd rlvnnmh
$ ls
27340 gcpcffp.fqg
$ cd ..
$ cd tmf
$ ls
193196 fgpnnvm
22126 gwftf.wcr
92461 hdh
50807 qbdmzjd.jvg
dir qrrmhwn
dir rdcsmpfm
dir rgl
36742 zsgdbd.dmm
$ cd qrrmhwn
$ ls
dir jgqzqhdc
109318 vtrtz.zvh
dir zmbtd
$ cd jgqzqhdc
$ ls
dir zsdbppb
$ cd zsdbppb
$ ls
325921 fgpnnvm
156452 tlcs.vzz
$ cd ..
$ cd ..
$ cd zmbtd
$ ls
93641 bpzttjt
dir grncq
dir hdh
7832 shl.gbz
95398 twnj.fsd
$ cd grncq
$ ls
dir qshl
$ cd qshl
$ ls
312490 stqg.gwj
$ cd ..
$ cd ..
$ cd hdh
$ ls
225454 sfhrzzcd.hwm
265069 twnj.hfs
$ cd ..
$ cd ..
$ cd ..
$ cd rdcsmpfm
$ ls
72657 dvqsffcn
308572 fgpnnvm
dir hdh
dir lqblqtng
180474 lsgjgsrb.jfm
67774 ptpjjwc.bqn
$ cd hdh
$ ls
19858 sfhrzzcd.hwm
$ cd ..
$ cd lqblqtng
$ ls
142164 cjvph.fvc
dir jpbh
dir prwnwvp
dir qbnthms
dir sthfhjf
$ cd jpbh
$ ls
167969 fgpnnvm
182107 jfzrww.tgb
$ cd ..
$ cd prwnwvp
$ ls
dir cczhv
70167 dvqsffcn
dir grncq
dir mbjwpdb
dir nll
dir qjjgjzbl
101178 qtqgcj
113714 sfhrzzcd.hwm
$ cd cczhv
$ ls
158391 fmwzpjdm.nwz
$ cd ..
$ cd grncq
$ ls
238901 hdh
188528 mjwcd.bqt
61037 wpfg.shs
$ cd ..
$ cd mbjwpdb
$ ls
dir bphnpft
72033 fgpnnvm
dir hdh
dir qdmjnqc
dir rplsvs
85983 tpfjp
303863 wzwvbnw.blb
$ cd bphnpft
$ ls
274331 gmmmlsj.pwp
$ cd ..
$ cd hdh
$ ls
154778 szs.dtw
$ cd ..
$ cd qdmjnqc
$ ls
133639 grncq.wnj
$ cd ..
$ cd rplsvs
$ ls
332906 zpstcbj.zvt
$ cd ..
$ cd ..
$ cd nll
$ ls
82368 cjvph.fvc
278280 ctmnzd
dir hdh
dir lqblqtng
211682 pshd.fds
319207 sfhrzzcd.hwm
46873 vrf.fgg
$ cd hdh
$ ls
dir hdtg
$ cd hdtg
$ ls
19779 cjvph.fvc
$ cd ..
$ cd ..
$ cd lqblqtng
$ ls
301792 dmhslp.hvz
305639 dvqsffcn
321833 strqtwrw
112471 twnj.fbn
20731 zsgdbd.dmm
$ cd ..
$ cd ..
$ cd qjjgjzbl
$ ls
78166 sztm.zrz
$ cd ..
$ cd ..
$ cd qbnthms
$ ls
158365 bvwjvrvr.bvb
$ cd ..
$ cd sthfhjf
$ ls
239051 dvqsffcn
dir ghgcfwg
327346 glfcswq.cqc
dir tjqhqs
124072 twnj.cbn
261890 zsgdbd.dmm
28917 zvbn.rbz
$ cd ghgcfwg
$ ls
dir szs
$ cd szs
$ ls
29912 fgpnnvm
$ cd ..
$ cd ..
$ cd tjqhqs
$ ls
125555 twnj.dnv
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd rgl
$ ls
dir grncq
59380 szs.jvh
dir wjjzprnz
$ cd grncq
$ ls
283850 lqblqtng.cdf
$ cd ..
$ cd wjjzprnz
$ ls
339632 hbg.vpp
15887 zglntj.qtt
$ cd ..
$ cd ..
$ cd ..
$ cd twjtnhll
$ ls
207430 cjvph.fvc
dir dnv
dir hdh
232881 wwp
$ cd dnv
$ ls
215966 lqblqtng.tdf
39850 szb.fqn
45436 twnj
$ cd ..
$ cd hdh
$ ls
dir grrs
288906 hvjv
219092 mnr.qhg
336470 rwh
145474 sfhrzzcd.hwm
24384 szs.fwn
dir twnj
$ cd grrs
$ ls
94496 dvqsffcn
$ cd ..
$ cd twnj
$ ls
dir sbhbnsw
$ cd sbhbnsw
$ ls
162705 sfhrzzcd.hwm
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd pgvmpmn
$ ls
18511 sfhrzzcd.hwm
$ cd ..
$ cd pqqcvcm
$ ls
201925 lqblqtng.mwb
300811 scjclwwz.bng
$ cd ..
$ cd zglbptq
$ ls
dir brhmv
237267 grncq
dir hdh
dir mhnfrn
131601 qtjwj
69185 tdtgmgw.qwc
96764 vppqqpf.znc
$ cd brhmv
$ ls
dir ddd
dir dpnghvpp
102457 grncq
239654 jtvzhvd.jvm
276919 sfhrzzcd.hwm
235776 zsgdbd.dmm
$ cd ddd
$ ls
dir grncq
dir mbrcgcd
$ cd grncq
$ ls
110323 fgpnnvm
$ cd ..
$ cd mbrcgcd
$ ls
100317 pbb.hcb
$ cd ..
$ cd ..
$ cd dpnghvpp
$ ls
259305 bscmwtd.mlw
dir hdh
$ cd hdh
$ ls
49708 grncq.zdl
dir szs
$ cd szs
$ ls
225139 fgpnnvm
258801 twnj.nrj
dir wwzpvb
$ cd wwzpvb
$ ls
4309 qlddd.cps
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd ..
$ cd hdh
$ ls
24775 hdh.htz
238695 jzpvhl
$ cd ..
$ cd mhnfrn
$ ls
30250 sfhrzzcd.hwm
dir twnj
$ cd twnj
$ ls
244061 qddtlr.mpf
"""